In [1]:
import sqlite3
import pandas as pd
import time
import requests
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam
import json
import difflib
import numpy as np

conexion = sqlite3.connect("NBA_DATA.db")

cursor = conexion.cursor()


In [4]:
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam
teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season='2016',
                                                        season_type_all_star='Regular Season',
                                                        team_id='1610612745').get_normalized_json()
teamGames

'{"CumeStatsTeamGames": []}'

In [2]:
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

def getSeasonScheduleFrame(season, seasonType, teamLookup):
    def getGameDate(matchup):
        return matchup.partition(' at')[0][:10]

    def getHomeTeam(matchup):
        return matchup.partition(' at')[2]

    def getAwayTeam(matchup):
        return matchup.partition(' at')[0][10:]

    def getTeamIDFromNickname(nickname):
        return teamLookup.loc[teamLookup['TEAM_NICKNAME'] == difflib.get_close_matches(nickname, teamLookup['TEAM_NICKNAME'], 1)[0]].values[0][0]

    @retry
    def getRegularSeasonSchedule(season, teamID, seasonType):
        season = str(season) + "-" + str(season + 1)[-2:]
        teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season=season,
                                                        season_type_all_star=seasonType,
                                                        team_id=teamID).get_normalized_json()

        teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
        teamGames['SEASON'] = season
        return teamGames

    scheduleFrame = pd.DataFrame()

    for id in teamLookup['TEAM_ID']:
        time.sleep(10)
        teamGames = getRegularSeasonSchedule(season, id, seasonType)
        scheduleFrame = pd.concat([scheduleFrame, teamGames], ignore_index=True)

    scheduleFrame['GAME_DATE'] = pd.to_datetime(scheduleFrame['MATCHUP'].map(getGameDate))
    scheduleFrame['HOME_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getHomeTeam)
    scheduleFrame['HOME_TEAM_ID'] = scheduleFrame['HOME_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame['AWAY_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getAwayTeam)
    scheduleFrame['AWAY_TEAM_ID'] = scheduleFrame['AWAY_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame = scheduleFrame.drop_duplicates()  # There's a row for both teams, only need 1
    scheduleFrame = scheduleFrame.reset_index(drop=True)

    return scheduleFrame

In [41]:
def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
        gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                            season=seasonYear,season_type_all_star="Regular Season",
                                            team_id = teamID).get_normalized_json()

        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME'] = awayTeamNickname.strip()
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

def getGameLogs(gameLogs,scheduleFrame):
    
    # Functions to prepare additional columns after gameLogs table loads
    def getHomeAwayFlag(gameDF):
        gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME']==1) | (gameDF['L_HOME']==1),1,0)
        gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD']==1) | (gameDF['L_ROAD']==1),1,0)
        #return gameDF 

    def getTotalWinPctg(gameDF):
        gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True)
        gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
        gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
        return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)

    def getHomeWinPctg(gameDF):
        gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
        gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
        gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
        return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)

    def getAwayWinPctg(gameDF):
        gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
        gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
        gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
        return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

    def getRollingOE(gameDF):
        gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

    def getRollingScoringMargin(gameDF):
        gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

    def getRestDays(gameDF):
        gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['GAME_DATE'].shift(1)
        gameDF['NUM_REST_DAYS'] = (gameDF['GAME_DATE'] - gameDF['LAST_GAME_DATE'])/np.timedelta64(1,'D') 
        return gameDF.drop('LAST_GAME_DATE',axis=1)
    
    def getmetrics(gameLogs):
        getHomeAwayFlag(gameLogs)
        gameLogs = getHomeWinPctg(gameLogs)
        gameLogs = getAwayWinPctg(gameLogs)
        gameLogs = getTotalWinPctg(gameLogs)
        getRollingScoringMargin(gameLogs)
        getRollingOE(gameLogs)
        gameLogs = getRestDays(gameLogs)

        return gameLogs
    
    start = time.perf_counter_ns()

    i = 0 #Can use a previously completed gameLog dataset

    while i<len(scheduleFrame):


        time.sleep(10)
        gameLogs = pd.concat([gameLogs, getSingleGameMetrics(scheduleFrame.at[i,'GAME_ID'],scheduleFrame.at[i,'H_TEAM_ID'],
                        scheduleFrame.at[i,'A_TEAM_ID'],scheduleFrame.at[i,'A_TEAM_NICKNAME'],
                        scheduleFrame.at[i,'SEASON'],scheduleFrame.at[i,'GAME_DATE'])])
        
        gameLogs = gameLogs.reset_index(drop=True)

        end = time.perf_counter_ns()

        if i%100 == 0:
            mins = ((end-start)/1e9)/60
            print(f"{i} games procesed in: {int(mins)} minutes")

        i += 1

    return getmetrics(gameLogs).reset_index(drop=True)

In [25]:
conexion = sqlite3.connect("NBA_DATA.db")

# Crear un cursor
cursor = conexion.cursor()

# Eliminar la tabla GAMES si ya existe
cursor.execute("DROP TABLE IF EXISTS GAMES")

# Confirmar los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()

In [26]:
conexion = sqlite3.connect("NBA_DATA.db")

cursor = conexion.cursor()

cursor.execute("""
create table If Not Exists GAMES (
GAME_ID text primary key,
GAME_DATE date not null,
H_TEAM_NICKNAME text not null,
A_TEAM_NICKNAME text not null,
H_TEAM_ID int references teams (TEAM_ID),
A_TEAM_ID int references teams (TEAM_ID),
SEASON int not null
)
""")

In [ ]:
season = 2021
seasonType = "Regular Season"
teamLookup = pd.read_sql("SELECT * FROM TEAMS", conexion)
scheduleFrame = getSeasonScheduleFrame(season, seasonType, teamLookup)

# Get the games already in the database
cursor.execute("SELECT GAME_ID FROM GAMES")
existing_game_ids = set(row[0] for row in cursor.fetchall())

# Get and save just the new games
new_games = scheduleFrame[~scheduleFrame['GAME_ID'].isin(existing_game_ids)].rename(columns={"HOME_TEAM_NICKNAME": "H_TEAM_NICKNAME", "HOME_TEAM_ID": "H_TEAM_ID",
                                            "AWAY_TEAM_NICKNAME": "A_TEAM_NICKNAME", "AWAY_TEAM_ID": "A_TEAM_ID"}).drop(columns='MATCHUP')
new_games.to_sql("GAMES", conexion, if_exists="append", index=False)

In [42]:
cursor.execute("""
create table If Not Exists GAME_STATS (
GAME_ID str references games (GAME_ID),
HOME_FLAG boolean not null,
AWAY_FLAG boolean not null,
CITY text,
NICKNAME text,
TEAM_ID int,
W int,
L int,
W_HOME int,
L_HOME int,
W_ROAD int,
L_ROAD int,
TEAM_TURNOVERS int,
TEAM_REBOUNDS int,
GP int,
GS int,
ACTUAL_MINUTES int,
ACTUAL_SECONDS int,
FG int,
FGA int,
FG_PCT double precision,
FG3 int,
FG3A int,
FG3_PCT double precision,
FT int,
FTA int,
FT_PCT double precision,
OFF_REB int,
DEF_REB int,
TOT_REB int,
AST int,
PF int,
STL int,
TOTAL_TURNOVERS int,
BLK int,
PTS int,
AVG_REB double precision,
AVG_PTS double precision,
DQ int,
OFFENSIVE_EFFICIENCY double precision,
SCORING_MARGIN double precision,
HOME_WIN_PCTG double precision,
AWAY_WIN_PCTG double precision,
TOTAL_WIN_PCTG double precision,
ROLLING_SCORING_MARGIN int,
ROLLING_OE double precision,
NUM_REST_DAYS int,
primary key (GAME_ID, HOME_FLAG)
)
""")

In [97]:
def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
        gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                            season=seasonYear,season_type_all_star="Regular Season",
                                            team_id = teamID).get_normalized_json()

        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME'] = awayTeamNickname.strip()
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

def getHomeAwayFlag(gameDF):
    gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME']==1) | (gameDF['L_HOME']==1),1,0)
    gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD']==1) | (gameDF['L_ROAD']==1),1,0)
    #return gameDF 

def getTotalWinPctg(gameDF):
    gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True)
    gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
    gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
    return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)

def getHomeWinPctg(gameDF):
    gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
    gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
    gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
    return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)

def getAwayWinPctg(gameDF):
    gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
    gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
    gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
    return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

def getRollingOE(gameDF):
    gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

def getRollingScoringMargin(gameDF):
    gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

def getRestDays(gameDF):
    gameDF['GAME_DATE'] = pd.to_datetime(gameDF['GAME_DATE'])
    gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['GAME_DATE'].shift(1)
    gameDF['NUM_REST_DAYS'] = (gameDF['GAME_DATE'] - gameDF['LAST_GAME_DATE'])/np.timedelta64(1,'D') 
    return gameDF.drop('LAST_GAME_DATE',axis=1)

def getmetrics(gameLogs):
    getHomeAwayFlag(gameLogs)
    gameLogs = getHomeWinPctg(gameLogs)
    gameLogs = getAwayWinPctg(gameLogs)
    gameLogs = getTotalWinPctg(gameLogs)
    getRollingScoringMargin(gameLogs)
    getRollingOE(gameLogs)
    gameLogs = getRestDays(gameLogs)

    return gameLogs

In [112]:
gameLogs = pd.DataFrame()

season_str = f'{season}-{season+1-2000}'
query = f"""
SELECT * 
FROM GAMES 
WHERE SEASON = '{season_str}' AND GAME_ID NOT IN (SELECT GAME_ID FROM GAME_STATS)
"""
games_without_stats = pd.read_sql_query(query, conexion)
print(len(games_without_stats))
if len(games_without_stats) > 0:
    scheduleFrame = games_without_stats
    start = time.perf_counter_ns()

    i = 0 #Can use a previously completed gameLog dataset

    while i<len(scheduleFrame):


        time.sleep(10)
        gameLogs = pd.concat([gameLogs, getSingleGameMetrics(scheduleFrame.at[i,'GAME_ID'],scheduleFrame.at[i,'H_TEAM_ID'],
                        scheduleFrame.at[i,'A_TEAM_ID'],scheduleFrame.at[i,'A_TEAM_NICKNAME'],
                        scheduleFrame.at[i,'SEASON'],scheduleFrame.at[i,'GAME_DATE'])])
        
        gameLogs = gameLogs.reset_index(drop=True)

        end = time.perf_counter_ns()

        if i%100 == 0:
            mins = ((end-start)/1e9)/60
            print(f"{i} games procesed in: {int(mins)} minutes")

        i += 1

    gameLogs = getmetrics(gameLogs).reset_index(drop=True).drop(columns=['GAME_DATE', 'SEASON'])
gameLogs.to_csv('gameLogs.csv')
gameLogs.to_sql("GAME_STATS", conexion, if_exists="append", index=False)

5
0 games procesed in: 0 minutes


10

In [110]:
gameLogs.to_sql("GAME_STATS", conexion, if_exists="append", index=False)

2450

In [109]:
gameLogs = gameLogs.drop(columns=['GAME_DATE', 'SEASON'])

In [107]:
gameLogs = gameLogs.iloc[:-10]

In [103]:
query = f"""
SELECT * 
FROM GAME_STATS 
"""
games_without_stats = pd.read_sql_query(query, conexion)
games_without_stats

,GAME_ID,HOME_FLAG,AWAY_FLAG,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,...,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,HOME_WIN_PCTG,AWAY_WIN_PCTG,TOTAL_WIN_PCTG,ROLLING_SCORING_MARGIN,ROLLING_OE,NUM_REST_DAYS


In [105]:
columns_games_without_stats = set(games_without_stats.columns)
columns_gameLogs = set(gameLogs.columns)

# Encontrar las columnas que están en games_without_stats pero no en gameLogs
columns_only_in_gameLogs = columns_gameLogs - columns_games_without_stats
columns_only_in_gameLogs

{'GAME_DATE', 'SEASON'}

In [13]:
from nba_api.stats.endpoints import leaguegamefinder
from datetime import datetime
import pandas as pd
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam
import json

teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season='2023-24',
                                                            season_type_all_star='Regular Season',
                                                            team_id=1610612757).get_normalized_json()

teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])

# Mostrar los partidos de hoy
teamGames.tail(40)

,MATCHUP,GAME_ID
42,01/17/2024 Nets at Trail Blazers,0022300578
43,01/14/2024 Suns at Trail Blazers,0022300554
44,01/12/2024 Trail Blazers at Timberwolves,0022300538
45,01/11/2024 Trail Blazers at Thunder,0022300529
46,01/09/2024 Trail Blazers at Knicks,0022300514
47,01/07/2024 Trail Blazers at Nets,0022300498
48,01/05/2024 Trail Blazers at Mavericks,0022300487
49,01/03/2024 Trail Blazers at Mavericks,0022300472
50,01/01/2024 Trail Blazers at Suns,0022300456
51,12/29/2023 Spurs at Trail Blazers,0022300437


In [9]:
gameStats = cumestatsteam.CumeStatsTeam(game_ids='0022401019',league_id ="00",
                                                season='2024-25',season_type_all_star="Regular Season",
                                                team_id = 1610612765).get_normalized_json()

gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])
gameStats

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,TOT_REB,AST,PF,STL,TOTAL_TURNOVERS,BLK,PTS,AVG_REB,AVG_PTS,DQ
0,Detroit,Pistons,1610612765,0,1,0,0,0,1,0,...,48,24,22,14,14,5,117,48.0,117.0,0
1,OPPONENTS,,0,1,0,1,0,0,0,0,...,56,25,16,10,20,7,123,56.0,123.0,1


In [16]:
from generate_database import database_inicialization
database_inicialization()

{'Oklahoma City Thunder': 1610612760, 'Boston Celtics': 1610612738, 'Cleveland Cavaliers': 1610612739, 'Los Angeles Lakers': 1610612747, 'Denver Nuggets': 1610612743, 'Golden State Warriors': 1610612744, 'New York Knicks': 1610612752, 'Milwaukee Bucks': 1610612749, 'Memphis Grizzlies': 1610612763, 'Minnesota Timberwolves': 1610612750, 'Los Angeles Clippers': 1610612746, 'Indiana Pacers': 1610612754, 'Houston Rockets': 1610612745, 'Detroit Pistons': 1610612765, 'Sacramento Kings': 1610612758, 'Chicago Bulls': 1610612741, 'Phoenix Suns': 1610612756, 'Orlando Magic': 1610612753, 'Atlanta Hawks': 1610612737, 'Miami Heat': 1610612748, 'Toronto Raptors': 1610612761, 'Philadelphia 76ers': 1610612755, 'Portland Trail Blazers': 1610612757, 'San Antonio Spurs': 1610612759, 'Brooklyn Nets': 1610612751, 'Dallas Mavericks': 1610612742, 'Charlotte Hornets': 1610612766, 'Washington Wizards': 1610612764}
Generation season schedule and game logs for season 2010


KeyboardInterrupt: 